# a

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for i in range(20):
    model.add(keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'))
model.add(keras.layers.Dense(10,activation='softmax'))

# b

In [3]:
from keras.datasets.cifar10 import load_data
(X_train,y_train),(X_test,y_test)=load_data()

In [4]:
X_train.shape,X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
X_valid = X_train[:5000]
y_valid = y_train[:5000]
X_train = X_train[5000:]
y_train = y_train[5000:]

In [6]:
X_train.shape

(45000, 32, 32, 3)

让我们使用学习率为 5e-5 的 Nadam 优化器。 我尝试了学习率 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3 和 1e-2，我比较了它们各自 10 个时期的学习曲线（使用下面的 TensorBoard 回调） . 3e-5 和 1e-4 的学习率都不错，所以我尝试了 5e-5，结果稍微好一些。

In [7]:
optimizer=keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [8]:
import os
early_stopping_cb=keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb=keras.callbacks.ModelCheckpoint("my_cifar10_model.h5",save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb=keras.callbacks.TensorBoard(run_logdir)

In [9]:
# model.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=[early_stopping_cb,checkpoint_cb,tensorboard_cb])

In [10]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 10s 2ms/step - loss: 1.5001 - accuracy: 0.4682


[1.5000526905059814, 0.4681999981403351]

# c

下面的代码与上面的代码非常相似，但有一些变化：

* 我在每个 Dense 层之后（在激活函数之前）都添加了一个 BN 层，除了输出层。 我还在第一个隐藏层之前添加了一个 BN 层。
* 我将学习率更改为 5e-4。 我对 1e-5、3e-5、5e-5、1e-4、3e-4、5e-4、1e-3 和 3e-3 进行了实验，我选择了 20 个 epoch 后验证性能最好的那个。
* 我将运行目录重命名为 run_bn_*，将模型文件名重命名为 my_cifar10_bn_model.h5。

In [11]:
import tensorflow as tf
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [12]:
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for i in range(20):
    model.add(keras.layers.Dense(100,kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))

model.add(keras.layers.Dense(10,activation='softmax'))

In [13]:
optimizer=keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [14]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath="my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, "my_cifar10_logs",
                          "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, checkpoint_cb, tensorboard_cb]

In [15]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 31s 12ms/step - loss: 1.9766 - accuracy: 0.2926 - val_loss: 1.6808 - val_accuracy: 0.3984
Epoch 2/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6820 - accuracy: 0.4024 - val_loss: 1.6186 - val_accuracy: 0.4276
Epoch 3/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6129 - accuracy: 0.4267 - val_loss: 1.5566 - val_accuracy: 0.4374
Epoch 4/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5548 - accuracy: 0.4477 - val_loss: 1.5179 - val_accuracy: 0.4614
Epoch 5/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5061 - accuracy: 0.4672 - val_loss: 1.4386 - val_accuracy: 0.4904
Epoch 6/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4639 - accuracy: 0.4795 - val_loss: 1.4183 - val_accuracy: 0.4960
Epoch 7/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4304 - accuracy: 0.4940 - val_loss: 1

In [16]:
model=keras.models.load_model('./my_cifar10_bn_model.h5')

In [17]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 2ms/step - loss: 1.3282 - accuracy: 0.5415


[1.3282344341278076, 0.5414999723434448]

* *模型的收敛速度是否比以前更快？* 快得多！ 之前的模型用了 27 个 epoch 才达到最低的验证损失，而新模型只用了 5 个 epoch 就达到了同样的损失，并且一直持续到第 16 个 epoch。 BN 层稳定了训练并允许我们使用更大的学习率，因此收敛速度更快。
* *BN 生产更好的模型吗？* 是的！ 最终模型也好得多，准确率为 54.0%，而不是 47.6%。 它仍然不是一个很好的模型，但至少它比以前好得多（卷积神经网络会做得更好，但这是一个不同的主题，参见第 14 章）。
* *BN 如何影响训练速度？* 尽管模型收敛得更快，但由于 BN 层需要额外的计算，每个 epoch 需要大约 12 秒而不是 8 秒。 但总体而言，训练时间（挂墙时间）显着缩短了！

# d

In [18]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [19]:
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for i in range(20):
    model.add(keras.layers.Dense(100,activation='selu',kernel_initializer='lecun_normal'))
    
model.add(keras.layers.Dense(10,activation='softmax'))

In [20]:
optimizer=keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [21]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [22]:
# 需要对数据集进行归一化。
X_means=X_train.mean(axis=0)
X_stds=X_train.std(axis=0)
X_train_scaled=(X_train-X_means)/X_stds
X_valid_scaled=(X_valid-X_means)/X_stds
X_test_scaled=(X_test-X_means)/X_stds

In [23]:
model.fit(X_train_scaled,y_train,validation_data=(X_valid_scaled,y_valid),epochs=100,callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 15s 8ms/step - loss: 2.0387 - accuracy: 0.2782 - val_loss: 1.8474 - val_accuracy: 0.3482
Epoch 2/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.7161 - accuracy: 0.3896 - val_loss: 1.7947 - val_accuracy: 0.3474
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6161 - accuracy: 0.4299 - val_loss: 1.7130 - val_accuracy: 0.3970
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5369 - accuracy: 0.4610 - val_loss: 1.6063 - val_accuracy: 0.4456
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4822 - accuracy: 0.4813 - val_loss: 1.5612 - val_accuracy: 0.4598
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4276 - accuracy: 0.5006 - val_loss: 1.5162 - val_accuracy: 0.4764
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.3845 - accuracy: 0.5161 - val_loss: 1.5164 -

In [24]:
model=keras.models.load_model('./my_cifar10_selu_model.h5')

In [25]:
model.evaluate(X_test_scaled,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.4576 - accuracy: 0.4968


[1.457561731338501, 0.4968000054359436]

我们得到了 47.9% 的准确率，这并不比原始模型（47.6%）好多少，也不如使用批量归一化的模型（54.0%）好。 然而，收敛速度几乎和 BN 模型一样快，而且每个 epoch 只需要 7 秒。 因此，它是迄今为止训练速度最快的模型。

# e